<a href="https://colab.research.google.com/github/Satty-hub/Eniac-project/blob/main/Brands_value_and_sales_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sales and revenue data by brands

In [ ]:
import pandas as pd

I have loaded all the clean CSV.data

In [ ]:
# orders_cl.csv
url = "https://drive.google.com/file/d/1Tla62vfu__kCqvgypZyVt2S9VuC016yH/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders_cl = pd.read_csv(path)

# orderlines_cl.csv
url = "https://drive.google.com/file/d/1OhtkQS2fwOYdzfd-qPh7im35iLc-L9TA/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines_cl = pd.read_csv(path)

# products_cl.csv
url = "https://drive.google.com/file/d/1s7Lai4NSlsYjGEPg1QSOUJobNYVsZBOJ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products_cl = pd.read_csv(path)
# brands_cl.csv
url = "https://drive.google.com/file/d/1WUh_PHzgEcUcwAR5kchA6iwJB_M9EXSw/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
brands_cl = pd.read_csv(path)

First make a copy and define the pandas df format

In [ ]:
# Make a copy for all the data
orders_qu = orders_cl.copy()
orderlines_qu = orderlines_cl.copy()
products_qu = products_cl.copy()
brands_qu = brands_cl .copy()

I want to check which brands sales the most. First I need to merge the products and orderlines df to get the brands information.

In [ ]:
orders_qu.head(1)

,order_id,created_date,total_paid,state
0,241319,2017-01-02 13:35:40,44.99,Cancelled


In [ ]:
orderlines_qu.head(5)

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38


In [ ]:
products_qu.head(1)

,sku,name,desc,price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,1,8696


In [ ]:
brands_qu.head(2)

,short,long
0,8MO,8Mobility
1,ACM,Acme


To check the brand name and quantity, price category we should first merged orderlines on sku and product quantity then rename the brands name with sku code then merged brands with orderlines merged. it will give table with short code product quantity and brands name

In [ ]:
orderlines_products = orderlines_qu.merge(products_qu, how="left", on = "sku")[["id_order","sku","name" ]]
orderlines_products.sample(5)

,id_order,sku,name
182684,489694,OWC0013,OWC Kit of 5 tools for Mac and PC
172175,477089,IFX0174,"Battery iFixit MacBook Pro 15 ""Retina Mid 2012..."
56278,354504,WDT0316,"Blue WD Hard Drive 1TB 35 ""Mac and PC"
151318,453213,JBL0097,JBL GO Bluetooth Speaker Blue
111202,410443,APP1190,IPhone AppleCare Protection Plan


In [ ]:
# First we should group the orderlines with sku and product_quantity
grouped_df = orderlines_qu.groupby(orderlines_qu['sku'].str [:3])['product_quantity'].sum().sort_values(ascending=False).reset_index()
grouped_df.rename(columns={'sku':'short'}, inplace = True)
products_brands = grouped_df.merge(brands_qu, how = 'left', on= 'short').sort_values (by = 'product_quantity', ascending= False)
products_brands.head(5)


,short,product_quantity,long
0,APP,55118,Apple
1,OWC,12069,OWC
2,WDT,9821,Western Digital
3,PAC,8004,Pack
4,CRU,7955,Crucial


In [ ]:
# Step : Group by 'brand' and sum up total revenue
# 1 . We have to merge the prodduct_brands data with ordelines.
orderlines_qu["short"] = orderlines_qu["sku"].str[:3]
products_brands_ol_df = products_brands.merge(orderlines_qu, how = 'left', on= 'short')
products_brands_ol_df.head(2)


,short,product_quantity_x,long,id,id_order,product_id,product_quantity_y,sku,unit_price,date
0,APP,55118,Apple,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11
1,APP,55118,Apple,1119140,299559,0,1,APP0700,72.19,2017-01-01 02:32:22


In [ ]:
# Step : Count total revenue for
# Step - Group first group 'brand' and sum up total revenue
brands_revenue = products_brands_ol_df.groupby ("long") ["unit_price"].sum().reset_index()

In [ ]:
# Step 3: Sort by product sale (descending order)
brands_revenue = brands_revenue.sort_values(by="unit_price", ascending=False)


In [ ]:
# Revenue from top 20 brands
top_20_brands = brands_revenue.head(20)
top_20_brands

,long,unit_price
6,Apple,16844258.95
109,Pack,2357725.49
102,OWC,1386749.49
69,LaCie,1292031.38
166,Western Digital,1104677.91
67,LG,1082772.40
165,Wacom,997390.71
21,Crucial,921752.40
26,Dell,793504.60
148,Synology,787049.99


In [ ]:
#  Select the top 20 brands
total_revenue = products_brands_ol_df["unit_price"].sum()
total_revenue

np.float64(35602569.629999995)

In [ ]:
# Calculate % of top 20 brands total revenue from all brands
top_20_brands["percentage"] = (top_20_brands["unit_price"]/total_revenue)*100
print("\nTop 20 Brands Revenue & Percentage Contribution:")
print(top_20_brands)


Top 20 Brands Revenue & Percentage Contribution:
                long   unit_price  percentage
6              Apple  16844258.95   47.311919
109             Pack   2357725.49    6.622346
102              OWC   1386749.49    3.895083
69             LaCie   1292031.38    3.629040
166  Western Digital   1104677.91    3.102804
67                LG   1082772.40    3.041276
165            Wacom    997390.71    2.801457
21           Crucial    921752.40    2.589005
26              Dell    793504.60    2.228785
148         Synology    787049.99    2.210655
120             QNAP    604721.68    1.698534
130          Samsung    566382.12    1.590846
17              Bose    389877.78    1.095083
133          Seagate    386510.13    1.085624
11              BenQ    346537.21    0.973349
94         NewerTech    245076.13    0.688366
140            Sonos    236258.21    0.663599
10            Belkin    212746.51    0.597559
40      G-Technology    199472.43    0.560275
131          SanDisk    186742

<ipython-input-15-62f6405f0c42>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_20_brands["percentage"] = (top_20_brands["unit_price"]/total_revenue)*100


In [ ]:
#  Count total Apple products sale
apple_pr_counts = products_brands[products_brands['long'] == 'Apple']
apple_pr_counts

,short,product_quantity,long
0,APP,55118,Apple
17,AP2,3092,Apple


In [ ]:
orderlines_qu

,id,id_order,product_id,product_quantity,sku,unit_price,date,short
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,OTT
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,LGE
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,PAR
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,WDT
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,JBL
...,...,...,...,...,...,...,...,...
216245,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,JBL
216246,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,PAC
216247,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,APP
216248,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,BEZ


In [ ]:
apple_sales_count = products_brands[products_brands['long'] == 'Apple'] ['product_quantity'].sum()
apple_pr_counts

,short,product_quantity,long
0,APP,55118,Apple
17,AP2,3092,Apple


In [ ]:
brands_qu

,short,long
0,8MO,8Mobility
1,ACM,Acme
2,ADN,Adonit
3,AII,Aiino
4,AKI,Akitio
...,...,...
182,XOO,Xoopar
183,XRI,X-Rite
184,XTO,Xtorm
185,ZAG,ZaggKeys


In [ ]:
#1. create a column called 'short' so you can merge brand with it
#2. merge brand table on short, so you retain apple
#3. find all the apple products, where the long == Apple

In [ ]:
# orderlines_cl["short"] = orderlines_cl["sku"].str[:3]
# ol_brands = orderlines_cl.merge(brands, on="short", how="left")
# apple_products = ol_brands.loc[ol_brands["long"] == "Apple"]
# apple_products.merge(products_cl, on= 'sku', how="left")

In [ ]:
#1. create a column called 'short' so you can merge brand with it
orderlines_qu["short"] = orderlines_qu["sku"].str[:3]

In [ ]:
orderlines_qu

,id,id_order,product_id,product_quantity,sku,unit_price,date,short
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,OTT
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,LGE
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,PAR
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,WDT
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,JBL
...,...,...,...,...,...,...,...,...
216245,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,JBL
216246,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,PAC
216247,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,APP
216248,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,BEZ


In [ ]:
#2. merge brand table on short, so you retain apple
order_apple_brands = orderlines_qu.merge(brands_qu , on = "short" , how= "left")
order_apple_brands

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,OTT,Otterbox
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,LGE,LG
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,PAR,Parrot
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,WDT,Western Digital
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,JBL,JBL
...,...,...,...,...,...,...,...,...,...
216245,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,JBL,JBL
216246,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,PAC,Pack
216247,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,APP,Apple
216248,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,BEZ,Be.ez


In [ ]:
#2. Find the apple products from the new table where long= Apple
apple_p_list = order_apple_brands.loc[order_apple_brands["long"] == "Apple"]
apple_p_list

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long
8,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11,APP,Apple
18,1119140,299559,0,1,APP0700,72.19,2017-01-01 02:32:22,APP,Apple
19,1119143,299559,0,1,APP1214,84.99,2017-01-01 02:36:43,APP,Apple
22,1119153,299562,0,1,APP1045,9.00,2017-01-01 02:39:53,APP,Apple
27,1119162,299567,0,1,APP1647,769.00,2017-01-01 03:16:51,APP,Apple
...,...,...,...,...,...,...,...,...,...
216234,1650184,527389,0,1,APP0698,9.99,2018-03-14 13:52:15,APP,Apple
216236,1650187,527390,0,1,APP0698,9.99,2018-03-14 13:52:33,APP,Apple
216240,1650193,527394,0,1,APP1922,161.00,2018-03-14 13:55:15,APP,Apple
216247,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,APP,Apple


In [ ]:
#2. How many types of Apple product we have. We need to group the apple products with products df.

apple_p_df= apple_p_list.merge(products_qu, on= 'sku', how="left")
apple_p_df

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type
0,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11,APP,Apple,Apple 85W MagSafe 2 charger MacBook Pro screen...,Apple MagSafe 2 Charger for MacBook Pro 15-inc...,89.0,1.0,13005399
1,1119140,299559,0,1,APP0700,72.19,2017-01-01 02:32:22,APP,Apple,Apple 85W MagSafe 2 charger MacBook Pro screen...,Apple MagSafe 2 Charger for MacBook Pro 15-inc...,89.0,1.0,13005399
2,1119143,299559,0,1,APP1214,84.99,2017-01-01 02:36:43,APP,Apple,Apple Magic Mouse 2,Bluetooth Apple Wireless Mouse,89.0,1.0,1387
3,1119153,299562,0,1,APP1045,9.00,2017-01-01 02:39:53,APP,Apple,Apple USB-C to USB,USB-C USB adapter for MacBook.,25.0,1.0,12585395
4,1119162,299567,0,1,APP1647,769.00,2017-01-01 03:16:51,APP,Apple,Apple iPhone 32GB Rose Gold 7,New Apple iPhone 32GB Rose Gold 7 Free,639.0,0.0,85641716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,1650184,527389,0,1,APP0698,9.99,2018-03-14 13:52:15,APP,Apple,Apple Lightning Cable Connector to USB 1m Whit...,Apple Lightning USB Cable 1 meter to charge an...,25.0,1.0,1230
52556,1650187,527390,0,1,APP0698,9.99,2018-03-14 13:52:33,APP,Apple,Apple Lightning Cable Connector to USB 1m Whit...,Apple Lightning USB Cable 1 meter to charge an...,25.0,1.0,1230
52557,1650193,527394,0,1,APP1922,161.00,2018-03-14 13:55:15,APP,Apple,AirPods Apple Bluetooth Headset for iPhone iPa...,Apple AirPods wireless headsets and cargo tran...,179.0,1.0,5384
52558,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,APP,Apple,Apple Lightning Cable Connector to USB 1m Whit...,Apple Lightning USB Cable 1 meter to charge an...,25.0,1.0,1230


Category creation by search term.
Let's start by creating a column category. For now we'll fill this column with a blank string "".

In [ ]:
apple_p_df["category"] = ""
apple_p_df.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type,category
0,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11,APP,Apple,Apple 85W MagSafe 2 charger MacBook Pro screen...,Apple MagSafe 2 Charger for MacBook Pro 15-inc...,89.0,1.0,13005399,
1,1119140,299559,0,1,APP0700,72.19,2017-01-01 02:32:22,APP,Apple,Apple 85W MagSafe 2 charger MacBook Pro screen...,Apple MagSafe 2 Charger for MacBook Pro 15-inc...,89.0,1.0,13005399,
2,1119143,299559,0,1,APP1214,84.99,2017-01-01 02:36:43,APP,Apple,Apple Magic Mouse 2,Bluetooth Apple Wireless Mouse,89.0,1.0,1387,
3,1119153,299562,0,1,APP1045,9.00,2017-01-01 02:39:53,APP,Apple,Apple USB-C to USB,USB-C USB adapter for MacBook.,25.0,1.0,12585395,
4,1119162,299567,0,1,APP1647,769.00,2017-01-01 03:16:51,APP,Apple,Apple iPhone 32GB Rose Gold 7,New Apple iPhone 32GB Rose Gold 7 Free,639.0,0.0,85641716,


We can find all the products with certain words in their description using .loc[] and .str.contains(). Here we'll look at all the items that have the word keyboard in their description.

In [ ]:
apple_p_df.loc[apple_p_df["desc"].str.contains("keyboards" , case= False, na=False)]
apple_p_df


,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type,category
0,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11,APP,Apple,Apple 85W MagSafe 2 charger MacBook Pro screen...,Apple MagSafe 2 Charger for MacBook Pro 15-inc...,89.0,1.0,13005399,
1,1119140,299559,0,1,APP0700,72.19,2017-01-01 02:32:22,APP,Apple,Apple 85W MagSafe 2 charger MacBook Pro screen...,Apple MagSafe 2 Charger for MacBook Pro 15-inc...,89.0,1.0,13005399,
2,1119143,299559,0,1,APP1214,84.99,2017-01-01 02:36:43,APP,Apple,Apple Magic Mouse 2,Bluetooth Apple Wireless Mouse,89.0,1.0,1387,
3,1119153,299562,0,1,APP1045,9.00,2017-01-01 02:39:53,APP,Apple,Apple USB-C to USB,USB-C USB adapter for MacBook.,25.0,1.0,12585395,
4,1119162,299567,0,1,APP1647,769.00,2017-01-01 03:16:51,APP,Apple,Apple iPhone 32GB Rose Gold 7,New Apple iPhone 32GB Rose Gold 7 Free,639.0,0.0,85641716,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,1650184,527389,0,1,APP0698,9.99,2018-03-14 13:52:15,APP,Apple,Apple Lightning Cable Connector to USB 1m Whit...,Apple Lightning USB Cable 1 meter to charge an...,25.0,1.0,1230,
52556,1650187,527390,0,1,APP0698,9.99,2018-03-14 13:52:33,APP,Apple,Apple Lightning Cable Connector to USB 1m Whit...,Apple Lightning USB Cable 1 meter to charge an...,25.0,1.0,1230,
52557,1650193,527394,0,1,APP1922,161.00,2018-03-14 13:55:15,APP,Apple,AirPods Apple Bluetooth Headset for iPhone iPa...,Apple AirPods wireless headsets and cargo tran...,179.0,1.0,5384,
52558,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,APP,Apple,Apple Lightning Cable Connector to USB 1m Whit...,Apple Lightning USB Cable 1 meter to charge an...,25.0,1.0,1230,


In [ ]:
apple_p_df["category"].value_counts()

,count
category,
,52560


In [ ]:
apple_p_df.loc[apple_p_df["desc"].str.contains("keyboards" , case= False, na=False), "category"] = "keyboard"


We can also use a product's name to select products for our categories.

In [ ]:
apple_p_df.loc[apple_p_df["name"].str.contains("apple iphone", case=False, na=False)]

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type,category
4,1119162,299567,0,1,APP1647,769.00,2017-01-01 03:16:51,APP,Apple,Apple iPhone 32GB Rose Gold 7,New Apple iPhone 32GB Rose Gold 7 Free,639.0,0.0,85641716,
6,1119196,299583,0,1,APP1648,879.00,2017-01-01 08:54:20,APP,Apple,Apple iPhone 7 128GB Black,New Apple iPhone 7 Free Black 128GB,749.0,0.0,85641716,
7,1119211,299593,0,1,APP1651,879.00,2017-01-01 09:40:11,APP,Apple,Apple iPhone 7 128GB Gold Rosa,New Apple iPhone 7 128GB Rose Gold free,749.0,0.0,85641716,
14,1119277,299618,0,1,APP1651,879.00,2017-01-01 11:13:03,APP,Apple,Apple iPhone 7 128GB Gold Rosa,New Apple iPhone 7 128GB Rose Gold free,749.0,0.0,85641716,
15,1119291,299626,0,3,APP1646,769.00,2017-01-01 11:37:23,APP,Apple,Apple iPhone 32GB Gold 7,New Apple iPhone 32GB 7 Free Gold,639.0,0.0,85641716,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52301,1649462,527051,0,1,APP1647,619.00,2018-03-14 11:43:41,APP,Apple,Apple iPhone 32GB Rose Gold 7,New Apple iPhone 32GB Rose Gold 7 Free,639.0,0.0,85641716,
52339,1649552,527090,0,1,APP2549,43.00,2018-03-14 11:53:47,APP,Apple,Apple iPhone Silicone Case X Case White,Silicone Case Apple official,45.0,1.0,11865403,
52480,1649975,527296,0,1,APP2489,908.27,2018-03-14 12:58:26,APP,Apple,Apple iPhone 64GB Space Gray X,New Free iPhone 64GB X,1159.0,1.0,113271716,
52492,1650018,527310,0,1,AP20406,459.00,2018-03-14 13:05:42,AP2,Apple,Apple iPhone 6S Plus 16GB Space Gray Apple Cer...,iPhone 6S Plus Free 16GB Apple Certified Refur...,812.0,1.0,24811716,


In [ ]:
apple_p_iphone = apple_p_df[apple_p_df['long'] == 'Apple'].groupby("unit_price").agg({"product_quantity": "count"})
apple_p_list

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long
8,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11,APP,Apple
18,1119140,299559,0,1,APP0700,72.19,2017-01-01 02:32:22,APP,Apple
19,1119143,299559,0,1,APP1214,84.99,2017-01-01 02:36:43,APP,Apple
22,1119153,299562,0,1,APP1045,9.00,2017-01-01 02:39:53,APP,Apple
27,1119162,299567,0,1,APP1647,769.00,2017-01-01 03:16:51,APP,Apple
...,...,...,...,...,...,...,...,...,...
216234,1650184,527389,0,1,APP0698,9.99,2018-03-14 13:52:15,APP,Apple
216236,1650187,527390,0,1,APP0698,9.99,2018-03-14 13:52:33,APP,Apple
216240,1650193,527394,0,1,APP1922,161.00,2018-03-14 13:55:15,APP,Apple
216247,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,APP,Apple


Looks like we get a lot of accessories included in this search. We can refine this using a little regex. Here, we will add .{0,7} at the beginning of the search: this means we will find all apple iphones that have 7 or less characters preceding the term "apple iphone" - if there's 8 characters preceding the search term, it won't be found. This should help refine our search by using the nomenclature of the DataFrame to our advantage.

If you feel unsure about regex, please use regex101. It's really useful for checking your code, and parts of other people's code that you're unsure about.

In [ ]:
# Creating category using regex
apple_p_df.loc[apple_p_df["desc"].str.contains("apple iphone", case=False, na=False)]

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type,category
4,1119162,299567,0,1,APP1647,769.00,2017-01-01 03:16:51,APP,Apple,Apple iPhone 32GB Rose Gold 7,New Apple iPhone 32GB Rose Gold 7 Free,639.0,0.0,85641716,
6,1119196,299583,0,1,APP1648,879.00,2017-01-01 08:54:20,APP,Apple,Apple iPhone 7 128GB Black,New Apple iPhone 7 Free Black 128GB,749.0,0.0,85641716,
7,1119211,299593,0,1,APP1651,879.00,2017-01-01 09:40:11,APP,Apple,Apple iPhone 7 128GB Gold Rosa,New Apple iPhone 7 128GB Rose Gold free,749.0,0.0,85641716,
11,1119261,299612,0,1,APP0927,19.99,2017-01-01 10:54:57,APP,Apple,EarPods Apple Headphones with Remote and Mic (...,EarPods headphones Apple iPhone iPad and iPod ...,35.0,1.0,5384,
14,1119277,299618,0,1,APP1651,879.00,2017-01-01 11:13:03,APP,Apple,Apple iPhone 7 128GB Gold Rosa,New Apple iPhone 7 128GB Rose Gold free,749.0,0.0,85641716,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52539,1650141,527365,0,1,APP0927,13.99,2018-03-14 13:38:34,APP,Apple,EarPods Apple Headphones with Remote and Mic (...,EarPods headphones Apple iPhone iPad and iPod ...,35.0,1.0,5384,
52542,1650150,527368,0,1,APP0927,13.99,2018-03-14 13:40:23,APP,Apple,EarPods Apple Headphones with Remote and Mic (...,EarPods headphones Apple iPhone iPad and iPod ...,35.0,1.0,5384,
52545,1650153,527369,0,1,APP0927,13.99,2018-03-14 13:40:39,APP,Apple,EarPods Apple Headphones with Remote and Mic (...,EarPods headphones Apple iPhone iPad and iPod ...,35.0,1.0,5384,
52552,1650178,527385,0,1,APP0927,13.99,2018-03-14 13:49:21,APP,Apple,EarPods Apple Headphones with Remote and Mic (...,EarPods headphones Apple iPhone iPad and iPod ...,35.0,1.0,5384,


In [ ]:
apple_iphone_list= apple_p_df.loc[apple_p_df["desc"].str.contains ("^.{0,7}apple iphone", case=False, na=False)]
apple_iphone_list.head(2)


,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type,category
4,1119162,299567,0,1,APP1647,769.0,2017-01-01 03:16:51,APP,Apple,Apple iPhone 32GB Rose Gold 7,New Apple iPhone 32GB Rose Gold 7 Free,639.0,0.0,85641716,
6,1119196,299583,0,1,APP1648,879.0,2017-01-01 08:54:20,APP,Apple,Apple iPhone 7 128GB Black,New Apple iPhone 7 Free Black 128GB,749.0,0.0,85641716,


In [ ]:
apple_iphone_list.loc[apple_iphone_list["desc"].str.contains("^.{0,7}apple iphone", case=False, na=False), "category"] = "iphone"

In [ ]:
apple_iphone_list["category"].value_counts()

,count
category,
iphone,8105


In [ ]:
apple_iphone_list['total_price'] = apple_iphone_list['unit_price'] * apple_iphone_list['product_quantity']
apple_iphone_list['total_price'].sum()

<ipython-input-36-d391c2149cca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apple_iphone_list['total_price'] = apple_iphone_list['unit_price'] * apple_iphone_list['product_quantity']


np.float64(6544754.37)

In [ ]:
apple_iphone_list[['total_price', "unit_price", "price"]].describe()

,total_price,unit_price,price
count,8105.000000,8105.000000,8105.000000
mean,807.495912,774.636665,802.157299
std,404.769046,152.833061,131.303296
min,0.000000,0.000000,409.000000
25%,734.000000,734.000000,749.000000
50%,793.000000,793.000000,809.000000
75%,890.020000,889.000000,919.000000
max,22672.000000,998.620000,1130.330000


In [ ]:
# Creating category using regex
ipad_list= apple_p_df.loc[apple_p_df["desc"].str.contains("apple ipad", case=False, na=False)]

In [ ]:
ipad_list

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type,category
16,1119301,299631,0,1,APP1749,429.00,2017-01-01 11:43:38,APP,Apple,Apple iPad Air 2 Wi-Fi 32GB Silver,New Apple iPad Air 2 Wi-Fi 32GB (MNV62TY / A).,429.00,0.0,42931714,
17,1119352,299652,0,1,APP1465,94.99,2017-01-01 12:25:44,APP,Apple,Spanish Keyboard Keyboard Magic Apple Mac (OEM),Spanish Keyboard Mac and Apple iPad Ultrathin ...,119.00,1.0,13855401,
34,1119557,299750,0,1,APP1759,344.62,2017-01-01 15:32:13,APP,Apple,Apple iPad Mini 4 Wi-Fi 32GB Gold,Apple iPad Mini 4 32GB Wi-Fi.,429.00,0.0,24861714,
35,1119562,299754,0,1,APP1759,416.99,2017-01-01 15:39:21,APP,Apple,Apple iPad Mini 4 Wi-Fi 32GB Gold,Apple iPad Mini 4 32GB Wi-Fi.,429.00,0.0,24861714,
57,1119958,299963,0,1,APP1221,106.00,2017-01-01 20:50:52,APP,Apple,Apple Pencil,Pencil Apple iPad Pro.,109.00,1.0,1229,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52140,1648869,526720,0,1,APP1120,382.65,2018-03-13 23:15:26,APP,Apple,Apple iPad Mini 4 Wi-Fi 128GB Gold,Apple iPad Wi-Fi Mini 4 128GB.,482.81,0.0,24861714,
52316,1649507,527066,0,1,APP1465,99.99,2018-03-14 11:48:15,APP,Apple,Spanish Keyboard Keyboard Magic Apple Mac (OEM),Spanish Keyboard Mac and Apple iPad Ultrathin ...,119.00,1.0,13855401,
52439,1649846,527235,0,13,APP1221,106.00,2018-03-14 12:34:46,APP,Apple,Apple Pencil,Pencil Apple iPad Pro.,109.00,1.0,1229,
52470,1649947,527278,0,1,APP1118,463.00,2018-03-14 12:52:44,APP,Apple,Apple iPad Wi-Fi Mini 4 128GB Space Gray,Apple iPad Wi-Fi Mini 4 128GB.,482.81,0.0,13621714,


In [ ]:
ipad_list['category'].value_counts()

,count
category,
,2565


In [ ]:
ipad_list

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type,category
16,1119301,299631,0,1,APP1749,429.00,2017-01-01 11:43:38,APP,Apple,Apple iPad Air 2 Wi-Fi 32GB Silver,New Apple iPad Air 2 Wi-Fi 32GB (MNV62TY / A).,429.00,0.0,42931714,
17,1119352,299652,0,1,APP1465,94.99,2017-01-01 12:25:44,APP,Apple,Spanish Keyboard Keyboard Magic Apple Mac (OEM),Spanish Keyboard Mac and Apple iPad Ultrathin ...,119.00,1.0,13855401,
34,1119557,299750,0,1,APP1759,344.62,2017-01-01 15:32:13,APP,Apple,Apple iPad Mini 4 Wi-Fi 32GB Gold,Apple iPad Mini 4 32GB Wi-Fi.,429.00,0.0,24861714,
35,1119562,299754,0,1,APP1759,416.99,2017-01-01 15:39:21,APP,Apple,Apple iPad Mini 4 Wi-Fi 32GB Gold,Apple iPad Mini 4 32GB Wi-Fi.,429.00,0.0,24861714,
57,1119958,299963,0,1,APP1221,106.00,2017-01-01 20:50:52,APP,Apple,Apple Pencil,Pencil Apple iPad Pro.,109.00,1.0,1229,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52140,1648869,526720,0,1,APP1120,382.65,2018-03-13 23:15:26,APP,Apple,Apple iPad Mini 4 Wi-Fi 128GB Gold,Apple iPad Wi-Fi Mini 4 128GB.,482.81,0.0,24861714,
52316,1649507,527066,0,1,APP1465,99.99,2018-03-14 11:48:15,APP,Apple,Spanish Keyboard Keyboard Magic Apple Mac (OEM),Spanish Keyboard Mac and Apple iPad Ultrathin ...,119.00,1.0,13855401,
52439,1649846,527235,0,13,APP1221,106.00,2018-03-14 12:34:46,APP,Apple,Apple Pencil,Pencil Apple iPad Pro.,109.00,1.0,1229,
52470,1649947,527278,0,1,APP1118,463.00,2018-03-14 12:52:44,APP,Apple,Apple iPad Wi-Fi Mini 4 128GB Space Gray,Apple iPad Wi-Fi Mini 4 128GB.,482.81,0.0,13621714,


In [ ]:
ipad_list.loc[ipad_list["desc"].str.contains("^.{0,7}apple ipad", case=False, na=False), "category"] = "ipad"

In [ ]:
ipad_list.head(2)

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type,category
16,1119301,299631,0,1,APP1749,429.00,2017-01-01 11:43:38,APP,Apple,Apple iPad Air 2 Wi-Fi 32GB Silver,New Apple iPad Air 2 Wi-Fi 32GB (MNV62TY / A).,429.0,0.0,42931714,ipad
17,1119352,299652,0,1,APP1465,94.99,2017-01-01 12:25:44,APP,Apple,Spanish Keyboard Keyboard Magic Apple Mac (OEM),Spanish Keyboard Mac and Apple iPad Ultrathin ...,119.0,1.0,13855401,


In [ ]:
# Total revenue generated by apple accessories product
ipad_total_revnue = (ipad_list['unit_price']* ipad_list['product_quantity']).sum()
ipad_total_revnue

np.float64(732401.24)

In [ ]:
# Total ipod product

In [ ]:
ipod_list= apple_p_df.loc[apple_p_df["name"].str.contains("apple ipod", case=False, na=False)]

In [ ]:
ipod_list

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type,category
26,1119460,299700,0,1,APP1098,169.99,2017-01-01 13:33:19,APP,Apple,Apple iPod Nano 16Gb Gold,Music player iPod Nano 16GB the 8th generation...,179.00,0.0,11821715,
28,1119470,299704,0,1,APP0697,12.99,2017-01-01 13:41:17,APP,Apple,OEM USB Cable for iPhone iPad Dock and Apple i...,Cable OEM dock for iPhone iPad and iPod compat...,25.00,1.0,1230,
30,1119497,299714,0,1,APP1094,219.99,2017-01-01 14:12:37,APP,Apple,Apple iPod Touch 16GB Blue,New 6th generation iPod Touch 16GB with 8 mega...,229.00,0.0,11821715,
66,1120145,300056,0,1,APP1104,52.99,2017-01-01 22:29:29,APP,Apple,Apple iPod Shuffle 2GB Gray Space,Music player iPod Shuffle 2GB and small size.,55.00,0.0,11821715,
78,1120311,300136,0,1,APP0697,12.99,2017-01-02 00:05:29,APP,Apple,OEM USB Cable for iPhone iPad Dock and Apple i...,Cable OEM dock for iPhone iPad and iPod compat...,25.00,1.0,1230,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51919,1646272,525516,0,1,APP1089,224.00,2018-03-11 18:06:38,APP,Apple,Apple iPod Touch 32GB Space Gray,New 6th generation iPod Touch 32GB with 8 mega...,232.81,0.0,11821715,
51984,1647041,525893,0,1,APP0985,16.99,2018-03-12 12:45:44,APP,Apple,OEM 5W USB Power Adapter iPhone iPod and compa...,OEM USB Power Adapter 5W 1A for iPhone iPad an...,25.00,1.0,14365395,
52219,1649247,526894,0,1,APP0985,16.99,2018-03-14 11:23:58,APP,Apple,OEM 5W USB Power Adapter iPhone iPod and compa...,OEM USB Power Adapter 5W 1A for iPhone iPad an...,25.00,1.0,14365395,
52253,1649330,526928,0,1,APP0697,6.99,2018-03-14 11:31:30,APP,Apple,OEM USB Cable for iPhone iPad Dock and Apple i...,Cable OEM dock for iPhone iPad and iPod compat...,25.00,1.0,1230,


In [ ]:
ipod_list.loc[ipod_list["name"].str.contains("^.{0,7}apple ipod", case=False, na=False), "category"] = "ipod"

In [ ]:
ipod_list

,id,id_order,product_id,product_quantity,sku,unit_price,date,short,long,name,desc,price,in_stock,type,category
26,1119460,299700,0,1,APP1098,169.99,2017-01-01 13:33:19,APP,Apple,Apple iPod Nano 16Gb Gold,Music player iPod Nano 16GB the 8th generation...,179.00,0.0,11821715,ipod
28,1119470,299704,0,1,APP0697,12.99,2017-01-01 13:41:17,APP,Apple,OEM USB Cable for iPhone iPad Dock and Apple i...,Cable OEM dock for iPhone iPad and iPod compat...,25.00,1.0,1230,
30,1119497,299714,0,1,APP1094,219.99,2017-01-01 14:12:37,APP,Apple,Apple iPod Touch 16GB Blue,New 6th generation iPod Touch 16GB with 8 mega...,229.00,0.0,11821715,ipod
66,1120145,300056,0,1,APP1104,52.99,2017-01-01 22:29:29,APP,Apple,Apple iPod Shuffle 2GB Gray Space,Music player iPod Shuffle 2GB and small size.,55.00,0.0,11821715,ipod
78,1120311,300136,0,1,APP0697,12.99,2017-01-02 00:05:29,APP,Apple,OEM USB Cable for iPhone iPad Dock and Apple i...,Cable OEM dock for iPhone iPad and iPod compat...,25.00,1.0,1230,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51919,1646272,525516,0,1,APP1089,224.00,2018-03-11 18:06:38,APP,Apple,Apple iPod Touch 32GB Space Gray,New 6th generation iPod Touch 32GB with 8 mega...,232.81,0.0,11821715,ipod
51984,1647041,525893,0,1,APP0985,16.99,2018-03-12 12:45:44,APP,Apple,OEM 5W USB Power Adapter iPhone iPod and compa...,OEM USB Power Adapter 5W 1A for iPhone iPad an...,25.00,1.0,14365395,
52219,1649247,526894,0,1,APP0985,16.99,2018-03-14 11:23:58,APP,Apple,OEM 5W USB Power Adapter iPhone iPod and compa...,OEM USB Power Adapter 5W 1A for iPhone iPad an...,25.00,1.0,14365395,
52253,1649330,526928,0,1,APP0697,6.99,2018-03-14 11:31:30,APP,Apple,OEM USB Cable for iPhone iPad Dock and Apple i...,Cable OEM dock for iPhone iPad and iPod compat...,25.00,1.0,1230,


In [ ]:
# Total ipod product
ipod_list['category'].value_counts()

,count
category,
,811
ipod,382


In [ ]:
# Total revenue generated by apple ipod product
ipod_total_revnue = (ipod_list['unit_price']* ipod_list['product_quantity']).sum()
ipod_total_revnue

np.float64(118651.84)

In [ ]:
Total_apple_revenue = (apple_p_df['unit_price']* apple_p_df['product_quantity']).sum()
Total_apple_revenue

np.float64(17954733.18)

We can find all the products with certain words in their description using .loc[] and .str.contains(). Here we'll look at all the items that have the word keyboard in their description.

In [ ]:
# your code here
orders_qu = orders_cl.copy()
orderlines_qu = orderlines_cl.copy()
products_qu = products_cl.copy()
brands_qu = brands_cl .copy()

In [ ]:
product_category_df = products_cl.copy()
brands_category_df = brands_cl.copy()

In [ ]:
product_category_df.head(2)

,sku,name,desc,price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,0,13855401


In [ ]:
brands_category_df.head(2)

,short,long
0,8MO,8Mobility
1,ACM,Acme


In [ ]:
LG_p_list = orderlines_brands.loc[orderlines_brands["long"] == "LG"]
LG_p_list

NameError: name 'orderlines_brands' is not defined

In [ ]:
# Step 1: Generate a list of product names (some specific, rest generic)
product_names = ["iPhone", "iPad", "iPod", "iMac"] + [f"Accessory_{i}" for i in (1, 97)]

In [ ]:
# Lets create a category from apple product with different type with if and else statements.
def categorize_product(apple_p_list):
    if apple_p_list in ["iphone" , "ipad" , "ipod" , "imac"]:
        return apple_p_list  # Keep their category as their name
    else:
       return "Accessory"  # All other items are accessories


In [ ]:
#  Create a Pandas DataFrame#
apple_p_df = pd.DataFrame ({'desc' : apple_p_list ['long']})
# Step 4: Apply categorization
apple_p_df['category'] = apple_p_df['desc'].apply(categorize_product)
apple_p_df.head()
print("\nCategory Counts:\n", apple_p_df['category'].value_counts())
print(apple_p_df['desc'].unique()[:20])


Category Counts:
 category
Accessory    52560
Name: count, dtype: int64
['Apple']


Category creation by search term
Let's start by creating a column category. For now we'll fill this column with a blank string "".

In [ ]:
#2. How many types of Apple product we have. We need to group the apple products with products df.

LG_p_df = LG_p_list.merge(products_qu, on= 'sku', how="left")
LG_p_df

NameError: name 'LG_p_list' is not defined

We can find all the products with certain words in their description using .loc[] and .str.contains(). Here we'll look at all the items that have the word keyboard in their description.

Next, we change the value in the category column to keyboard for all of these keyboard products.

In [ ]:
product_category_df.loc[product_category_df["desc"].str.contains("keyboard", case=False), "category"]= "keyboard"

Let's take a look at the effect that had on the category column.